# 爬蟲複習

In [1]:
import requests

URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

resp = requests.get(URL)
resp.text

'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>批踢踢實業坊</title>\n\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-common.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-base.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-custom.css">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/pushstream.css" media="screen">\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.22/bbs-print.css" media="print">\n\n\n\n\n\t</head>\n    <body>\n\t\t\n<div class="bbs-screen bbs-content">\n    <div class="over18-notice">\n        <p>本網站已依網站內容分級規定處理</p>\n\n        <p>警告︰您即將進入之看板內容需滿十八歲方可瀏覽。</p>\n\n        <p>若您尚未年滿十八歲，請點選離開。若您已滿十八歲，亦不可將本區之內容派發、傳閱、出售、出租、交給或借予年齡未滿18歲的人士瀏覽，或將本網站內容向該人士出示、播放或放映。</p>\n    </div>\n</div>\n\n<div class="bbs-screen bbs-content cent

## 仔細觀察瀏覽PTT八卦版的流程，並且打開開發者工具

In [10]:
import requests

URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

custom_headers = {
    "cookie": "over18=1;"
}

resp = requests.get(URL, headers=custom_headers)
resp.text[1500:1900]
# 成功！

'tn wide disabled">下頁 &rsaquo;</a>\n\t\t\t\t<a class="btn wide" href="/bbs/Gossiping/index.html">最新</a>\n\t\t\t</div>\n\t\t</div>\n\t</div>\n\n\t<div class="r-list-container action-bar-margin bbs-screen">\n\t\t\n\t\t\n            \n        \n        \n\t\t<div class="r-ent">\n\t\t\t<div class="nrec"><span class="hl f2">3</span></div>\n\t\t\t<div class="mark"></div>\n\t\t\t<div class="title">\n\t\t\t\n\t\t\t\t<a href="/bbs/Gossiping/M.1514806260.A.'

## 思考一下PTT的換頁策略

我們需要知道現在最新的頁面是多少

![https://i.imgur.com/MBX0eET.png](https://i.imgur.com/MBX0eET.png)

可以用上次的re模組撰寫regular express來取得

In [55]:
import re

import requests

URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

custom_headers = {
    "cookie": "over18=1;"
}

resp = requests.get(URL, headers=custom_headers)

# 這個符號
# ‹-> &lsaquo;
total_page_cnt = int(re.findall('href="/bbs/Gossiping/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1

total_page_cnt

31008

In [60]:
# 整理成function
def get_total_page_cnt():
    URL = "https://www.ptt.cc/bbs/Gossiping/index.html"

    custom_headers = {
        "cookie": "over18=1;"
    }

    resp = requests.get(URL, headers=custom_headers)

    # 這個符號
    # ‹-> &lsaquo;
    total_page_cnt = int(re.findall('href="/bbs/Gossiping/index(\d+).html">&lsaquo; 上頁', resp.text)[0]) + 1
    return total_page_cnt

## 單頁爬蟲

In [75]:
import re

import requests

def get_list_page(url):
    HOST = "https://www.ptt.cc"
    
    custom_headers = {
        "cookie": "over18=1;"
    }

    resp = requests.get(URL, headers=custom_headers)
    links = re.findall('<a href="(/bbs/Gossiping/M.+\.html)">.+</a>', resp.text)
    for link in links:
        # Generator
        # https://wiki.python.org/moin/Generators
        yield HOST + link

In [77]:
for link in get_list_page('https://www.ptt.cc/bbs/Gossiping/index31008.html'):
    print(link)

https://www.ptt.cc/bbs/Gossiping/M.1514807382.A.0CA.html
https://www.ptt.cc/bbs/Gossiping/M.1514807473.A.511.html
https://www.ptt.cc/bbs/Gossiping/M.1514807474.A.985.html
https://www.ptt.cc/bbs/Gossiping/M.1514807510.A.E2C.html
https://www.ptt.cc/bbs/Gossiping/M.1514807521.A.342.html
https://www.ptt.cc/bbs/Gossiping/M.1514807566.A.84B.html
https://www.ptt.cc/bbs/Gossiping/M.1514807580.A.0BF.html
https://www.ptt.cc/bbs/Gossiping/M.1514807581.A.E1D.html
https://www.ptt.cc/bbs/Gossiping/M.1514807585.A.663.html
https://www.ptt.cc/bbs/Gossiping/M.1514807609.A.9EC.html
https://www.ptt.cc/bbs/Gossiping/M.1514807618.A.D7F.html
https://www.ptt.cc/bbs/Gossiping/M.1514807622.A.3E3.html
https://www.ptt.cc/bbs/Gossiping/M.1510415718.A.D77.html
https://www.ptt.cc/bbs/Gossiping/M.1514720863.A.84E.html
https://www.ptt.cc/bbs/Gossiping/M.1514336396.A.B25.html
https://www.ptt.cc/bbs/Gossiping/M.1514738804.A.4A0.html


In [80]:
import re

import requests

# 一次爬10頁
CRAWL_PAGE_CNT = 10
URL_TEMPLATE = "https://www.ptt.cc/bbs/Gossiping/index{}.html"

total_page_cnt = get_total_page_cnt()

for pg in range(total_page_cnt, total_page_cnt - CRAWL_PAGE_CNT, -1):
    url = URL_TEMPLATE.format(pg)
    for link in get_list_page(url):
        print(link)

https://www.ptt.cc/bbs/Gossiping/M.1514807382.A.0CA.html
https://www.ptt.cc/bbs/Gossiping/M.1514807473.A.511.html
https://www.ptt.cc/bbs/Gossiping/M.1514807474.A.985.html
https://www.ptt.cc/bbs/Gossiping/M.1514807510.A.E2C.html
https://www.ptt.cc/bbs/Gossiping/M.1514807521.A.342.html
https://www.ptt.cc/bbs/Gossiping/M.1514807566.A.84B.html
https://www.ptt.cc/bbs/Gossiping/M.1514807580.A.0BF.html
https://www.ptt.cc/bbs/Gossiping/M.1514807581.A.E1D.html
https://www.ptt.cc/bbs/Gossiping/M.1514807585.A.663.html
https://www.ptt.cc/bbs/Gossiping/M.1514807609.A.9EC.html
https://www.ptt.cc/bbs/Gossiping/M.1514807618.A.D7F.html
https://www.ptt.cc/bbs/Gossiping/M.1514807622.A.3E3.html
https://www.ptt.cc/bbs/Gossiping/M.1514807702.A.BB6.html
https://www.ptt.cc/bbs/Gossiping/M.1514807707.A.12A.html
https://www.ptt.cc/bbs/Gossiping/M.1514807718.A.FC3.html
https://www.ptt.cc/bbs/Gossiping/M.1514807725.A.B98.html
https://www.ptt.cc/bbs/Gossiping/M.1510415718.A.D77.html
https://www.ptt.cc/bbs/Gossipin

https://www.ptt.cc/bbs/Gossiping/M.1514807382.A.0CA.html
https://www.ptt.cc/bbs/Gossiping/M.1514807473.A.511.html
https://www.ptt.cc/bbs/Gossiping/M.1514807474.A.985.html
https://www.ptt.cc/bbs/Gossiping/M.1514807510.A.E2C.html
https://www.ptt.cc/bbs/Gossiping/M.1514807521.A.342.html
https://www.ptt.cc/bbs/Gossiping/M.1514807566.A.84B.html
https://www.ptt.cc/bbs/Gossiping/M.1514807580.A.0BF.html
https://www.ptt.cc/bbs/Gossiping/M.1514807581.A.E1D.html
https://www.ptt.cc/bbs/Gossiping/M.1514807585.A.663.html
https://www.ptt.cc/bbs/Gossiping/M.1514807609.A.9EC.html
https://www.ptt.cc/bbs/Gossiping/M.1514807618.A.D7F.html
https://www.ptt.cc/bbs/Gossiping/M.1514807622.A.3E3.html
https://www.ptt.cc/bbs/Gossiping/M.1514807702.A.BB6.html
https://www.ptt.cc/bbs/Gossiping/M.1514807707.A.12A.html
https://www.ptt.cc/bbs/Gossiping/M.1514807718.A.FC3.html
https://www.ptt.cc/bbs/Gossiping/M.1514807725.A.B98.html
https://www.ptt.cc/bbs/Gossiping/M.1510415718.A.D77.html
https://www.ptt.cc/bbs/Gossipin

In [81]:
!mkdir /tmp/ptt_data

In [94]:
import requests

DATA_PATH = '/tmp/ptt_data'

def dump_page(url):
    filename = "_".join(url.split('/')[-1].split('.')[:-1]) + '.html'
    
    custom_headers = {
        "cookie": "over18=1;"
    }

    resp = requests.get(url, headers=custom_headers)
    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return url

In [95]:
dump_page('https://www.ptt.cc/bbs/Gossiping/M.1514807718.A.FC3.html')

'https://www.ptt.cc/bbs/Gossiping/M.1514807718.A.FC3.html'

In [96]:
!ls /tmp/ptt_data/

M_1510415718_A_D77.html M_1514807521_A_342.html M_1514807702_A_BB6.html
M_1514336396_A_B25.html M_1514807566_A_84B.html M_1514807707_A_12A.html
M_1514720863_A_84E.html M_1514807580_A_0BF.html M_1514807718_A_FC3.html
M_1514738804_A_4A0.html M_1514807581_A_E1D.html M_1514807725_A_B98.html
M_1514807382_A_0CA.html M_1514807585_A_663.html M_1514807776_A_25C.html
M_1514807473_A_511.html M_1514807609_A_9EC.html M_1514807789_A_C9C.html
M_1514807474_A_985.html M_1514807618_A_D7F.html M_1514807802_A_C56.html
M_1514807510_A_E2C.html M_1514807622_A_3E3.html M_1514807817_A_59A.html


In [97]:
import re

import requests

# 一次爬10頁
CRAWL_PAGE_CNT = 10
URL_TEMPLATE = "https://www.ptt.cc/bbs/Gossiping/index{}.html"

total_page_cnt = get_total_page_cnt()

for pg in range(total_page_cnt, total_page_cnt - CRAWL_PAGE_CNT, -1):
    url = URL_TEMPLATE.format(pg)
    for link in get_list_page(url):
        print(link)
        dump_page(link)

https://www.ptt.cc/bbs/Gossiping/M.1514807382.A.0CA.html
https://www.ptt.cc/bbs/Gossiping/M.1514807473.A.511.html
https://www.ptt.cc/bbs/Gossiping/M.1514807474.A.985.html
https://www.ptt.cc/bbs/Gossiping/M.1514807510.A.E2C.html
https://www.ptt.cc/bbs/Gossiping/M.1514807521.A.342.html
https://www.ptt.cc/bbs/Gossiping/M.1514807566.A.84B.html
https://www.ptt.cc/bbs/Gossiping/M.1514807580.A.0BF.html
https://www.ptt.cc/bbs/Gossiping/M.1514807581.A.E1D.html
https://www.ptt.cc/bbs/Gossiping/M.1514807585.A.663.html
https://www.ptt.cc/bbs/Gossiping/M.1514807609.A.9EC.html
https://www.ptt.cc/bbs/Gossiping/M.1514807618.A.D7F.html
https://www.ptt.cc/bbs/Gossiping/M.1514807622.A.3E3.html
https://www.ptt.cc/bbs/Gossiping/M.1514807702.A.BB6.html
https://www.ptt.cc/bbs/Gossiping/M.1514807707.A.12A.html
https://www.ptt.cc/bbs/Gossiping/M.1514807718.A.FC3.html
https://www.ptt.cc/bbs/Gossiping/M.1514807725.A.B98.html
https://www.ptt.cc/bbs/Gossiping/M.1514807776.A.25C.html
https://www.ptt.cc/bbs/Gossipin

KeyboardInterrupt: 

# 以上為上次複習

# 再來我們開看看怎麼將HTML的內文整理成稍微結構化的資料格式

In [98]:
page = {
    "title": "[問卦] 大尾鱸鰻 vs 與神同行？",
    "content": "兩部我都看完了，覺得差不多好看！",
    "author": "chiafann (NAZZZ)",
    "pub_date": "Mon Jan  1 19:55:15 2018",
    "ip": "185.89.251.21"
}

In [100]:
!pip install beautifulsoup4 lxml

In [121]:
from bs4 import BeautifulSoup

with open('/tmp/ptt_data/M_1514336396_A_B25.html') as f:
    html = f.read()

soup = BeautifulSoup(html, 'lxml')

In [122]:
type(soup)

bs4.BeautifulSoup

In [123]:
dict_key = ['author', 'board', 'title', 'pub_date']

page = {k:tag.text for k, tag in zip(dict_key, soup.select('span.article-meta-value'))}
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

In [124]:
import re

page['ip'] = re.findall('來自: (\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', html)[0]
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'ip': '42.72.160.231',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

# 內文比較複雜

因為內文沒有被tag包住，可以直接select

所以只能弄刪去法, 把不是內文的東西刪掉

In [142]:
content = soup.select_one('div#main-content')

selectors_to_del = ['.article-metaline',
                   '.article-metaline-right',
                    'span.f2',
                    'div.push'
                   ]

for selector in selectors_to_del:
    [tag.extract() for tag in content.select(selector)]
content

<div class="bbs-screen bbs-content" id="main-content">

作者: nswitch (Nswitch) 看板: BigBanciao
標題: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊
時間: Wed Dec 27 08:42:37 2017


因朋友權限不足 是以代po
並請友版版主轉文章並置底
非常感謝您 真的很感謝

內容
請廣大的鄉民們幫忙或可轉至其他社群幫忙尋找

時間: 12/24 星期日12:40～14:00這段時間遺失

地點:新北市耶誕城市民廣場處

因為家人那天帶小孩去新北耶誕城廣場玩
在玩了旋轉木馬和飛椅之後
發現脖子上的鑽石銀項鍊遺失了
可能是鍊子斷掉了
（特色是鑽石會自然晃動）
後來有請大會廣播並再找了一天都沒找到
甚至從縫隙看水溝蓋裡
也至警局備案 但目前仍沒有找到

鑽石重量十分不到 一般市價幾千元
但那鑽石是由家人做成的生命寶石
對我們意義重大 我們遺失非常的難過
所以 真的能希望由廣大版友幫忙 能夠找回
若有在那裡有撿到或是您的親友有看到拾得的話
請Line: Nswitch
或可交至耶誕城大會服務台
也可送附近警察局再轉交給海山分局

若是可能
會用超放大鏡去看腰間刻的英文名和寶石編號
經確定是我們遺失的
我們會給您台幣一百萬元感謝您有好好地照顧與保護
並在新北廣場發1000份雞排感謝ptt fb 或其他社群的朋友/恩人

附上同款項鍊圖片和新北耶誕城遺失位置圖片連結
<a href="https://imgur.com/gallery/KPuIH" rel="nofollow" target="_blank">https://imgur.com/gallery/KPuIH</a>

--

</div>

In [143]:
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'ip': '42.72.160.231',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}

In [144]:
page['content'] = content.text.strip()

In [145]:
page

{'author': 'Kay731 (K少)',
 'board': 'Gossiping',
 'content': '作者: nswitch (Nswitch) 看板: BigBanciao\n標題: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊\n時間: Wed Dec 27 08:42:37 2017\n\n\n因朋友權限不足 是以代po\n並請友版版主轉文章並置底\n非常感謝您 真的很感謝\n\n內容\n請廣大的鄉民們幫忙或可轉至其他社群幫忙尋找\n\n時間: 12/24 星期日12:40～14:00這段時間遺失\n\n地點:新北市耶誕城市民廣場處\n\n因為家人那天帶小孩去新北耶誕城廣場玩\n在玩了旋轉木馬和飛椅之後\n發現脖子上的鑽石銀項鍊遺失了\n可能是鍊子斷掉了\n（特色是鑽石會自然晃動）\n後來有請大會廣播並再找了一天都沒找到\n甚至從縫隙看水溝蓋裡\n也至警局備案 但目前仍沒有找到\n\n鑽石重量十分不到 一般市價幾千元\n但那鑽石是由家人做成的生命寶石\n對我們意義重大 我們遺失非常的難過\n所以 真的能希望由廣大版友幫忙 能夠找回\n若有在那裡有撿到或是您的親友有看到拾得的話\n請Line: Nswitch\n或可交至耶誕城大會服務台\n也可送附近警察局再轉交給海山分局\n\n若是可能\n會用超放大鏡去看腰間刻的英文名和寶石編號\n經確定是我們遺失的\n我們會給您台幣一百萬元感謝您有好好地照顧與保護\n並在新北廣場發1000份雞排感謝ptt fb 或其他社群的朋友/恩人\n\n附上同款項鍊圖片和新北耶誕城遺失位置圖片連結\nhttps://imgur.com/gallery/KPuIH\n\n--',
 'ip': '42.72.160.231',
 'pub_date': 'Wed Dec 27 08:59:55 2017',
 'title': 'Fw: [協尋] 12/24下午新北耶誕城廣場遺失鑽石項鍊'}